## Features needed for training
1. Gas Price of the Transaction
2. Mean Gas Price of Transactions in the Last 10 Blocks
3.  Standard Deviation of Gas Price of Transactions in the
Last Ten Blocks
4. Mean Gas Price of Transactions by the same EOA (externally owned account)
5. Standard Deviation of Gas Price in Transactions by the
same EOA
6. Usage of Gas Tokens
7. Predicted Gas Price

In [118]:
from web3 import Web3
from tqdm import tqdm
import pandas as pd
import numpy as np

web3 = Web3(Web3.HTTPProvider("https://intensive-sly-mountain.quiknode.pro/a3f5256d7f2af6541d483cce3f1d49c94c01879e/"))
print(web3.is_connected())

True


In [119]:
df_insertion = pd.read_csv ('../data/insertion_attacks.csv', delimiter=',')
#print(df_insertion.describe())
#print(df_insertion)
print('# of insertion: ', len(df_insertion))

# of insertion:  196691


### Feature 1
Gas price of transaction

In [120]:
def get_transaction_gas_price_in_eth_by_sender_and_block_nr(block_number, sender, web3):
    block = web3.eth.get_block(block_number, full_transactions=True)
    
    for transaction in block.transactions:        
        if transaction["from"] == sender:
            print(transaction)
            return (transaction["gasPrice"]) / 10**18

get_transaction_gas_price_in_eth_by_sender_and_block_nr(5599805, '0xFF28319a7cD2136ea7283E7cDb0675B50AC29Dd2', web3)

AttributeDict({'blockHash': HexBytes('0x19d72da6dbb1533debd84a3792b65a556f04f29d21db4a45f9ed9e56e8e97279'), 'blockNumber': 5599805, 'from': '0xFF28319a7cD2136ea7283E7cDb0675B50AC29Dd2', 'gas': 850000, 'gasPrice': 9970000000, 'hash': HexBytes('0x7b2dd075eca1ed39a44c3d9dbb4776b78081cb446f936cf9f612093fef210cae'), 'input': HexBytes('0x4993594f0000000000000000000000009a0242b7a33dacbe40edb927834f96eb39f8fbcb000000000000000000000000238b7e54dfee4d8e98b8d1a78ab40dd94349bcfd0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000005572590000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000001bea22a95dc73a00ddc895620cfe259b0a7a93cf46cab77703fa9a577ca057310705251000c99f61b8c18c78e8bf16785c488411d80ca93c4e9d561ad61f857809'), 'nonce': 45594, 'to': '0x1DB9Baef62D4ad8a74E9051B1a4b49419Fa01573', 'transacti

9.97e-09

### Feature 2 and 3
- Mean Gas Price of Transactions in the Last 10 Blocks
- Standard Deviation of Gas Price of Transactions in the Last 10 Blocks

In [121]:
def get_mean_and_std_gas_price_of_last_n_blocks(last_n_blocks, curr_block, web3):
    web3.eth.get_block(curr_block)
    
    gas_prices = []
    for i in range(last_n_blocks):
        block = web3.eth.get_block(curr_block - i, full_transactions=True)
        
        for transaction in block.transactions:
            gas_prices.append(transaction["gasPrice"] / 10**18)
    return np.mean(gas_prices), np.std(gas_prices)

mean, std = get_mean_and_std_gas_price_of_last_n_blocks(10, 5599805, web3)
print(mean)
print(std)

KeyboardInterrupt: 

### Feature 4
- Mean Gas Price of Transactions by the same EOA (externally owned account)
- Standard Deviation of Gas Price in Transactions by the
same EOA

In [ ]:
# prepare data for insertion

def get_mean_and_std_gas_price_of_last_n_blocks_of_same_EOA(last_n_blocks, curr_block, eoa_address, web3):
    web3.eth.get_block(curr_block)
    
    gas_prices = []
    for i in range(last_n_blocks):
        block = web3.eth.get_block(curr_block - i, full_transactions=True)
        
        for transaction in block.transactions:
            if transaction["from"] == eoa_address:
                gas_prices.append((transaction["gasPrice"]) / 10**18)
    return np.mean(gas_prices), np.std(gas_prices)



mean, std = get_mean_and_std_gas_price_of_last_n_blocks_of_same_EOA(50, 5599805, '0xfF1b9745f68F84F036E5e92c920038d895FB701A', web3)

print(mean, std)

### Feature 6
Usage of gas tokens -> check if they use the gas token address

In [ ]:
def is_transaction_using_gas_token(block_number, address, web3):
    block = web3.eth.get_block(block_number, full_transactions=True)
    
    for transaction in block.transactions:        
        if transaction["from"] == address and transaction["to"] == '0x0000000000b3F879cb30FE243b4Dfee438691c04':
            return True
    return False

print(is_transaction_using_gas_token(5576747, '0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C', web3))

### Feature 7
Predicted gas price --> train modell for this

In [ ]:
def get_gas_prices_from_last_100_blocks(web3):
    curr_block = web3.eth.block_number
    
    gas_prices = []
    for i in range(100):
        block = web3.eth.get_block(curr_block - i, full_transactions=True)
        
        for transaction in block.transactions:
            gas_prices.append(transaction["gasPrice"] / 10**18)
    return gas_prices
            
print(web3.eth.block_number)
print(len(get_gas_prices_from_last_100_blocks(web3)))

19569416
